# Laboratorium pierwsze
## Wyszukiwanie wzorców w tekście

In [1]:
import numpy as np
import time

#### Zaimportujmy najważniejsze biblioteki:
 * numpy - do wyznaczania alfabetu funkcją np.unique()
 * time - do liczenia czasu

In [2]:
def readfile(name="in.in"):
    file = open(name, "r")
    txt1 = file.read()
    file.close()
    return txt1, "art"

### Zaimplementuje algorytmy wszukiwania wzorców
 * Naiwny
 * Automat skończony
 * KMP

Funkcja readfile - przyjmuje jako argument nazwę pliku do przeczytania, a zwraca zawartość pliku i wzorzec do przeszukania -> funkcja dostosowana do wyszukania art w pliku zawierającym ustawę(domyślnie in.in)

In [3]:
def naive(source, pattern):
    ans = []
    for i in range(0, len(source) - len(pattern)+1):
        if source[i:i+len(pattern)] == pattern:
            ans.append(i)
    return ans

Funkcja naive - przyjmuje tekst i wzorzec w nim do wyszukania, a zwraca indeksy wzorców w tekście -> wykonuje algorytm naiwny

In [4]:
def trans(pattern, alf):
    mov = [{} for i in range(len(pattern)+1)]
    for i in alf:
        mov[0][i] = 0
    mov[0][pattern[0]] = 1
    lps = 0
    for i in range(1, len(pattern)+1):
        for x in alf:
            mov[i][x] = mov[lps][x]
        if i < len(pattern):
            mov[i][pattern[i]] = i+1
            lps = mov[lps][pattern[i]]
    return mov

Funkcja trans - przyjmuje wzorzec i alfabet, a zwraca tablicę przejśc, potrzebną do alogrytmu automatu skończonego

In [5]:
def auto(source, pattern, alf=set(), mov=[]):
    if len(alf)==0:
        alf = set(np.unique([x for x in pattern]))
    if len(mov)==0:
        mov = trans(pattern, alf)
    q = 0
    ans = []
    for s in range(0, len(source)):
        if source[s] not in alf:
            q = 0
        else:
            q = mov[q][source[s]]
            if q == len(mov)-1:
                ans += [s+1-q]
    return ans

Funkcja auto - przyjmuje tekst, wzorzec, alfabet (sam uzupełnia w przypadku braku) i tablicę przejść (sam uzupełnia w przypadku braku), a zwraca indeksy wzorców w tekście -> wykonuje algorytm na automat skończony

In [6]:
def pre(pattern):
    pi = [0]
    k = 0
    for i in range(1, len(pattern)):
        while k > 0 and pattern[i] != pattern[k]:
            k = pi[k-1]
        if pattern[k] == pattern[i]:
            k = k+1
        pi.append(k)
    return pi

Funkcja pre - przyjmuje wzorzec i zwraca tablcę prefiksową, potrzebną do KMP

In [7]:
def kmp(source, pattern, ps=[]):
    if len(ps)==0:
        ps = pre(pattern)
    res = []
    q = 0
    for i in range(len(source)):
        while q > 0 and pattern[q] != source[i]:
            q = ps[q-1]
        if pattern[q] == source[i]:
            q += 1
        if q == len(pattern):
            res += [i+1-q]
            q = ps[q-1]
    return res

Funkcja kmp - przyjmuje tekst, wzorzec i tablicę prefiksową (automatycznie tworzy jeśli jej brak, zwraca indeksy wzorców w tekście -> wykonuje algorytm KMP

### Zaimplementuj testy porównujące szybkość działania wyżej wymienionych algorytmów
Zakładam, że chodzi o funkcje porównujące czasy.

In [8]:
def times(txt, pat):
    stna = time.process_time()
    resna = naive(txt, pat)
    enna = time.process_time()

    stau = time.process_time()
    resau = auto(txt, pat)
    enau = time.process_time()

    stkm = time.process_time()
    reskm = kmp(txt, pat)
    enkm = time.process_time()

    print(f"naive {enna-stna} {len(resna)}")
    print(f"automat {enau-stau} {len(resau)}")
    print(f"kmp {enkm-stkm} {len(reskm)}")

Funkcja times - przyjmuje tekst i wzorzec, wykonuje wszystkie 3 algorytmy i wypisuje ich czasy oraz liczbę wystąpień wzorca w tekście

In [9]:
def creation(pat):
    alf = set(np.unique([x for x in pat]))
    stpa = time.process_time()
    trans(pat, alf)
    enpa = time.process_time()

    stpk = time.process_time()
    pre(pat)
    enpk = time.process_time()

    print(f"automat {enpa-stpa}")
    print(f"kmp {enpk-stpk}")

Funkcja creation - przyjmuje wzorzec, tworzy obie tablice pomocnicze, dla automatu skończonego i kmp oraz wypisuje czas ich tworzenia

### Znajdź wszystkie wystąpienia "art" w ustawie, każdym algorytmem

In [10]:
txt, pat = readfile()
WN = naive(txt, pat)
WA = auto(txt, pat)
WK = kmp(txt, pat)
print(f"Naiwny: {len(WN)}")
print(f"Automat: {len(WA)}")
print(f"KMP: {len(WK)}")
print(f"Equal: {WN==WA==WK}")

Naiwny: 273
Automat: 273
KMP: 273
Equal: True


Można wypisać dowolną listę, ale ze względu na 273 elementów pozwoliłem sobie to ominąć, porównałem zawartości i liczebności wyników.

### Porównaj szybkości działania algorytmów dla poprzedniegoprzykładu

In [11]:
txt, pat = readfile()
times(txt, pat)

naive 0.0873694319999998 273
automat 0.04567884499999986 273
kmp 0.11448136899999994 273


Wzorzec jest bardzo krótki, więć różnice w czasie wykonania są bardzo nieznaczne, jednak podejście naiwne wychodzi najgorzej (tak jak możnaby oczekiwać)

### Porównaj czasy wyszukiwania "kruszwil" w wikipedii

In [12]:
def readwiki(file="wiki.txt", pat="kruszwil"):
    resna=[]
    stna = time.process_time()
    with open(file) as txt_file:
        for line in txt_file:
            resna += naive(line, pat)
    enna = time.process_time()

    resau=[]
    stau = time.process_time()
    alf = set(np.unique([x for x in pat]))
    mov = trans(pat, alf)
    with open(file) as txt_file:
        for line in txt_file:
            resau += auto(line, pat, alf=alf, mov=mov)
    enau = time.process_time()

    reskm=[]
    stkm = time.process_time()
    ps = pre(pat)
    with open(file) as txt_file:
        for line in txt_file:
            reskm += kmp(line, pat, ps=ps)
    enkm = time.process_time()

    print(f"naive {enna-stna} {len(resna)}")
    print(f"automat {enau-stau} {len(resau)}")
    print(f"kmp {enkm-stkm} {len(reskm)}")

Funkcja readwiki - przyjmuje jako argument nazwę pliku (domyślnie "wiki.txt") i wzorzec do wyszukania (domyślnie "kruszwil"), liczy czas i wypisuje czasy i liczby wyników, dla każdego algorytmu, przechodząc plik po jednej linii

In [13]:
readwiki()

naive 67.443785846 13
automat 49.494618493000004 13
kmp 62.969511069000006 13


Jak widać, już dla takiego przykładu podejście naiwne działa najwolniej. Co prawda różnica jest nieznaczna, ze względu małą liczbę wzorców w tekście i "nietypowe" słowo, którego początek rzadko kiedy występuje razem nawet w innych słowach

### Zaproponuj tekst oraz wzorzec, dla którego zmierzony czas działania algorytmów 2 oraz 3 będzie co najmniej 2-krotnie krótszy niż dla algorytmu naiwnego.

In [14]:
txt = "a"*500000
pat = "a"*50001
times(txt, pat)

naive 2.425235743999991 450000
automat 0.2840591689999883 450000
kmp 0.26842023099999324 450000


Dla tekstu zawierającego wzorzec dużo razy (albo prawie wzorzec), i długiego wzorca algorytmy kmp i automat skończony są dużo szybsze od podejścia naiwnego

### Zaproponuj wzorzec, dla którego zmierzony czas obliczenia tablicy przejścia automatu skończonego będzie co najmniej dwukrotnie dłuższy, niż czas potrzebny na utworzenie funkcji przejścia w algorytmie KMP.

In [15]:
pat = "abcdefghijklmnopqrstuvwxyz1234567890-=+[]_';/..,ABCDEFGHIJKLMNOPQRSTUVWXYZ!@#$%^&*()~`"
creation(pat)

automat 0.0012072780000096373
kmp 2.547599999047634e-05


Jak widać na przykładzie, czas utworzenia tablicy przejścia automatu skończonego jest mocno zależny o rozmiaru alfabetu, przez co na różnorodnych wzorcach, o dużych alfabetach będzie on konstruowany dłużej.